In [ ]:
import pysam
import pandas as pd 
import os 
import glob
from collections import defaultdict

In [ ]:
files = glob.glob("/SGRNJ06/randd/USER/cjj/celedev/rna/20231106/*/05.count/*matrix/genes.tsv")

In [ ]:
files

In [ ]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/rna/20231106/K562_18TL_bulk_NL/05.count/K562_18TL_bulk_NL_raw_feature_bc_matrix/genes.tsv", sep='\t',header=None)

In [ ]:
df.rename(columns = {0: "id", 1: "gene"})

In [ ]:
mt_set, rrna_set = set(), set()
for file in files:
    df = pd.read_csv(file, sep='\t', header=None)
    df = df.rename(columns = {0: "id", 1: "gene"})
    for i in set(df.gene):
        if i.startswith("MT-"):
            mt_set.add(i)
        elif i.startswith("RPS"):
            rrna_set.add(i)
        elif i.startswith("RPL"):
            rrna_set.add(i)

In [ ]:
df_mt = df[df['gene'].isin(mt_set)]

In [ ]:
df_rrna = df[df['gene'].isin(rrna_set)]

In [ ]:
mt_id = set(df_mt['id'])
rrna_id = set(df_rrna['id'])

In [ ]:
file = "/SGRNJ06/randd/USER/cjj/celedev/rna/20231106/K562_18T_bulk_NL/04.featureCounts/K562_18T_bulk_NL_aligned_posSorted_addTag.bam"

In [ ]:
bam = pysam.AlignmentFile(file, "rb")

In [ ]:
mt_read_count = 0
rrna_read_count = 0
total_read_count = 0
for read in bam:
    try:
        gene_id = read.get_tag("XT")
    except KeyError:
        gene_id = None
    if gene_id in mt_id:
        mt_read_count += 1
    elif gene_id in rrna_id:
        rrna_read_count += 1
    total_read_count += 1

In [ ]:
mt_read_count

In [ ]:
rrna_read_count

In [ ]:
total_read_count

In [ ]:
file = "/SGRNJ06/randd/USER/cjj/celedev/rna/20231106/K562_18TL_bulk_NL/04.featureCounts/K562_18TL_bulk_NL_aligned_posSorted_addTag.bam"

In [ ]:
bam = pysam.AlignmentFile(file, "rb")

In [ ]:
mt_read_count = 0
rrna_read_count = 0
total_read_count = 0
for read in bam:
    try:
        gene_id = read.get_tag("XT")
    except KeyError:
        gene_id = None
    if gene_id in mt_id:
        mt_read_count += 1
    elif gene_id in rrna_id:
        rrna_read_count += 1
    total_read_count += 1

In [ ]:
mt_read_count

In [ ]:
rrna_read_count

In [ ]:
total_read_count

In [158]:
res = defaultdict(int)

In [159]:
# 占比前十的gene
file = "/SGRNJ06/randd/USER/cjj/celedev/rna/20231106/K562_18TL_bulk_NL/04.featureCounts/K562_18TL_bulk_NL_aligned_posSorted_addTag.bam"
bam = pysam.AlignmentFile(file, "rb")
for read in bam:
    try:
        gene_id = read.get_tag("XT")
    except KeyError:
        gene_id = None
    res[gene_id] += 1

In [160]:
res = sorted(res.items(),key=lambda x:x[1],reverse=True)

In [161]:
top_gene_id = []
top_gene_id_count = []
for k in res:
    top_gene_id.append(k[0])
    top_gene_id_count.append(k[1])

In [162]:
top100_gene_id = top_gene_id[1:1001]
top100_gene_id_count = top_gene_id_count[1:1001]

In [163]:
df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/rna/20231106/K562_18TL_bulk_NL/05.count/K562_18TL_bulk_NL_raw_feature_bc_matrix/genes.tsv", sep='\t',header=None)
df = df.rename(columns = {0: "id", 1: "gene"})

In [164]:
id_gene_dict = dict(zip(list(df['id']), list(df['gene'])))

In [165]:
c={"id":top100_gene_id,
   'count':top100_gene_id_count}  #用列表a，b构造字典
 
data=pd.DataFrame(c)

In [166]:
data['id'] = data['id'].apply(lambda x: id_gene_dict[x])

In [167]:
data

,id,count
0,MT-RNR1,139975
1,MT-RNR2,63983
2,MT-CO3,38059
3,MT-CO1,36238
4,MALAT1,35973
...,...,...
995,MRPL18,824
996,ATXN7L3B,822
997,SELENOW,822
998,GYPC,822


In [169]:
data.to_csv("/SGRNJ06/randd/USER/cjj/celedev/rna/20231106/K562_18TL_bulk_NL_top1000.tsv", sep='\t')